In [29]:
import sys 
sys.path.append(
    "/Users/kirillklimushin/Desktop/news_classification_project"
)
sys.path

['/Users/kirillklimushin/Desktop/news_classification_project/experiments/experiment_1/notebooks',
 '/opt/homebrew/Cellar/python@3.10/3.10.12_1/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/opt/homebrew/Cellar/python@3.10/3.10.12_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/opt/homebrew/Cellar/python@3.10/3.10.12_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/Users/kirillklimushin/Desktop/news_classification_project/news_env/lib/python3.10/site-packages',
 '/Users/kirillklimushin/Desktop/new_classification_project/',
 '/Users/kirillklimushin/Desktop/new_classification_project',
 '/Users/kirillklimushin/Desktop/new_classification_project/src',
 '/Users/kirillklimushin/Desktop/new_classification_project/src',
 '/Users/kirillklimushin/Desktop/new_classification_project/src',
 '/Users/kirillklimushin/Desktop/news_classification_project/src',
 '/Users/kirillklimushin/Desktop/news_classification_project/src',
 '/Users/kir

In [40]:
import matplotlib.pyplot as plt 
import pandas
from src.drift import detection
import pickle
from constants import constants
from experiments import compress_dataset

In [31]:
%load_ext line_profiler 
%load_ext memory_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


Compressing Dataset

In [41]:
%run compress_dataset.py

Exception: File `'compress_dataset.py'` not found.

Defining constant values

In [20]:
NEW_DATA_URL = "../data/compressed_data/category_dataset.pkl"
OLD_DATA_URL = "../../experiment_%s/data/compressed_data/category_dataset.pkl" % str(constants.LAST_EXPERIMENT_NUMBER)

Loading dataset

In [ ]:
new_news_applications = pickle.load(open(NEW_DATA_URL, mode='rb'))
old_news_applications = pickle.load(open(OLD_DATA_URL, mode='rb'))

In [ ]:
new_news_applications

In [28]:
old_news_applications

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


Calculating statistics

In [29]:
stat_summary = detection.compare_datasets(
    old_data=old_news_applications,
    new_data=new_news_applications
)

Checking results

In [30]:
stat_summary

defaultdict(dict,
            {'link': {'p_value': None, 'null_exceed': True},
             'headline': {'p_value': None, 'null_exceed': True},
             'category': {'p_value': None, 'null_exceed': True},
             'short_description': {'p_value': None, 'null_exceed': True},
             'authors': {'p_value': None, 'null_exceed': True},
             'date': {'p_value': None, 'null_exceed': True}})

Comparing numerical distributions

In [34]:
def compare_distributions(old_dataset: pandas.DataFrame, new_dataset: pandas.DataFrame):
    """
    Function plots numerical distributions of old and new data
    using matplotlib library

    Args:
        old_dataset (pandas.DataFrame) old version of the original dataset
        new_dataset (pandas.DataFrame) new version of the original dataset
    """

    _, axs = plt.subplots(len(new_dataset.columns), 2)

    for idx, feature in enumerate(new_dataset.columns):
        
        axs[idx, 0].histplot(data=old_dataset[feature], marker='o')
        axs[idx, 0].set_title("Old " + feature + " Distribution")

        axs[idx, 1].histplot(data=new_dataset[feature], marker='o')
        axs[idx, 1].set_title("New " + feature + " Distribution")

    plt.title("Numerical Data Distributions")
    plt.show()

numerical_features = new_news_applications.select_dtypes(include='number').columns

# Visualizing distributions

compare_distributions(
    old_dataset=old_news_applications[numerical_features],
    new_dataset=new_news_applications[numerical_features]
)


ValueError: Number of rows must be a positive integer, not 0

<Figure size 640x480 with 0 Axes>